In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, f1_score,accuracy_score
from collections import Counter
import warnings
from pandasgui import show
import gc
import catboost as cb
import ipywidgets
from sklearn.ensemble import BaggingClassifier
from collections import Counter
from scipy import stats
warnings.filterwarnings("ignore")

In [4]:
attr = pd.read_csv('attr.txt', sep='\t',
                       names=['link', 'length', 'direction', 'path_class', 'speed_class', 'LaneNum', 'speed_limit',
                              'level', 'width'], header=None)

In [5]:
pro = pd.read_csv('per_road_pro.txt')

In [6]:
ver = pd.read_csv('train/is_train_20190730.txt')
#ver['is_night'] = ver['future_slice_id'].apply(lambda x: 0 if (0< x <= 172) | (x>=690) else 1)
ver = ver.merge(attr, on='link', how='left')
ver = ver.merge(pro, on='link', how='left')

In [7]:
use_cols = [i for i in ver.columns if i not in ['link', 'label', 'label_pred']]
X_ver = (ver.loc[:,use_cols])
y_ver = (ver['label'])

In [8]:
class_ = []

In [9]:
test = pd.read_csv('testgai.txt')

test = test.merge(pro, on='link', how='left')
X_test= (test.loc[:,use_cols])
y_test = (test['label'])

## 多个单天bagging

In [10]:
from sklearn.ensemble import BaggingClassifier
for i in tqdm(range(20,30)):
    path = 'train/is_train_201907'+str(i)+'.txt'
    data = pd.read_csv(path)
    #data['is_night'] = data['future_slice_id'].apply(lambda x: 0 if (0< x <= 172) | (x>=690) else 1)
    data = data.merge(attr, on='link', how='left')
    data = data.merge(pro, on='link', how='left')
    X_train= (data.loc[:,use_cols])
    y_train = (data['label'])
    tree = cb.CatBoostClassifier(iterations=100,depth=5,l2_leaf_reg=8,learning_rate=0.5,random_strength=1,losns_function='MultiClassOneVsAll',logging_level='Verbose')
    bag = BaggingClassifier(base_estimator=tree,n_estimators=6,max_samples=1.0,max_features=1.0,bootstrap=True,
                            bootstrap_features=False,n_jobs=1,random_state=1)
    w = np.array((y_train+1))
    bag = bag.fit(X_train,y_train,sample_weight=w)
    class_.append(bag)
    y_train_pred = bag.predict(X_train)
    y_test_pred = bag.predict(X_ver)
    test_pred = bag.predict(X_test)
    if (i==20):
        sum_ = y_test_pred
        sum_test = test_pred
        i = 0
    else:
        sum_ = np.vstack((sum_,y_test_pred))
        sum_test = np.vstack((sum_test,test_pred))
    print('sum_:',sum_.shape)
    bag_train = accuracy_score(y_train,y_train_pred)
    bag_test = accuracy_score(y_ver,y_test_pred)
    print('Bagging train/test accuracies %.3f/%.3f' % (bag_train,bag_test))

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

0:	learn: 0.4825355	total: 237ms	remaining: 23.5s
1:	learn: 0.4010285	total: 413ms	remaining: 20.2s
2:	learn: 0.3634096	total: 582ms	remaining: 18.8s
3:	learn: 0.3417035	total: 766ms	remaining: 18.4s
4:	learn: 0.3316040	total: 947ms	remaining: 18s
5:	learn: 0.3257875	total: 1.15s	remaining: 18s
6:	learn: 0.3219938	total: 1.32s	remaining: 17.5s
7:	learn: 0.3196029	total: 1.5s	remaining: 17.2s
8:	learn: 0.3175908	total: 1.67s	remaining: 16.9s
9:	learn: 0.3164681	total: 1.83s	remaining: 16.5s
10:	learn: 0.3150494	total: 1.99s	remaining: 16.1s
11:	learn: 0.3138079	total: 2.16s	remaining: 15.9s
12:	learn: 0.3133778	total: 2.3s	remaining: 15.4s
13:	learn: 0.3127713	total: 2.49s	remaining: 15.3s
14:	learn: 0.3120457	total: 2.68s	remaining: 15.2s
15:	learn: 0.3111870	total: 2.85s	remaining: 15s
16:	learn: 0.3102716	total: 3.06s	remaining: 15s
17:	learn: 0.3098129	total: 3.25s	remaining: 14.8s
18:	learn: 0.3094294	total: 3.43s	remaining: 14.6s
19:	learn: 0.3090387	total: 3.6s	remaining: 14.4s
2

62:	learn: 0.2939602	total: 11.5s	remaining: 6.73s
63:	learn: 0.2937711	total: 11.7s	remaining: 6.57s
64:	learn: 0.2936169	total: 11.9s	remaining: 6.4s
65:	learn: 0.2934122	total: 12s	remaining: 6.2s
66:	learn: 0.2932047	total: 12.2s	remaining: 6.03s
67:	learn: 0.2930828	total: 12.4s	remaining: 5.83s
68:	learn: 0.2928390	total: 12.6s	remaining: 5.66s
69:	learn: 0.2925338	total: 12.8s	remaining: 5.49s
70:	learn: 0.2923245	total: 13s	remaining: 5.31s
71:	learn: 0.2922027	total: 13.2s	remaining: 5.12s
72:	learn: 0.2919510	total: 13.3s	remaining: 4.93s
73:	learn: 0.2917183	total: 13.5s	remaining: 4.74s
74:	learn: 0.2915878	total: 13.7s	remaining: 4.56s
75:	learn: 0.2913529	total: 13.9s	remaining: 4.39s
76:	learn: 0.2911692	total: 14.1s	remaining: 4.21s
77:	learn: 0.2909489	total: 14.3s	remaining: 4.03s
78:	learn: 0.2907670	total: 14.5s	remaining: 3.85s
79:	learn: 0.2905765	total: 14.7s	remaining: 3.67s
80:	learn: 0.2902858	total: 14.9s	remaining: 3.49s
81:	learn: 0.2900493	total: 15s	remai

26:	learn: 0.3033096	total: 5.17s	remaining: 14s
27:	learn: 0.3028692	total: 5.39s	remaining: 13.9s
28:	learn: 0.3026053	total: 5.58s	remaining: 13.7s
29:	learn: 0.3021171	total: 5.79s	remaining: 13.5s
30:	learn: 0.3018020	total: 5.96s	remaining: 13.3s
31:	learn: 0.3015633	total: 6.16s	remaining: 13.1s
32:	learn: 0.3012561	total: 6.35s	remaining: 12.9s
33:	learn: 0.3011058	total: 6.51s	remaining: 12.6s
34:	learn: 0.3007374	total: 6.7s	remaining: 12.4s
35:	learn: 0.3005013	total: 6.89s	remaining: 12.3s
36:	learn: 0.3002646	total: 7.08s	remaining: 12s
37:	learn: 0.2999798	total: 7.25s	remaining: 11.8s
38:	learn: 0.2997783	total: 7.41s	remaining: 11.6s
39:	learn: 0.2996676	total: 7.57s	remaining: 11.4s
40:	learn: 0.2994785	total: 7.73s	remaining: 11.1s
41:	learn: 0.2992148	total: 7.89s	remaining: 10.9s
42:	learn: 0.2988650	total: 8.04s	remaining: 10.7s
43:	learn: 0.2986049	total: 8.21s	remaining: 10.4s
44:	learn: 0.2984159	total: 8.36s	remaining: 10.2s
45:	learn: 0.2983204	total: 8.5s	rem

89:	learn: 0.2893437	total: 16.2s	remaining: 1.8s
90:	learn: 0.2891222	total: 16.4s	remaining: 1.62s
91:	learn: 0.2889932	total: 16.6s	remaining: 1.44s
92:	learn: 0.2888495	total: 16.8s	remaining: 1.26s
93:	learn: 0.2886402	total: 17s	remaining: 1.08s
94:	learn: 0.2885100	total: 17.2s	remaining: 903ms
95:	learn: 0.2883899	total: 17.3s	remaining: 721ms
96:	learn: 0.2881176	total: 17.5s	remaining: 540ms
97:	learn: 0.2879430	total: 17.7s	remaining: 361ms
98:	learn: 0.2877870	total: 17.9s	remaining: 180ms
99:	learn: 0.2875719	total: 18s	remaining: 0us
0:	learn: 0.4817106	total: 184ms	remaining: 18.2s
1:	learn: 0.4000049	total: 361ms	remaining: 17.7s
2:	learn: 0.3642900	total: 535ms	remaining: 17.3s
3:	learn: 0.3439798	total: 710ms	remaining: 17s
4:	learn: 0.3320891	total: 890ms	remaining: 16.9s
5:	learn: 0.3255900	total: 1.08s	remaining: 16.9s
6:	learn: 0.3221755	total: 1.29s	remaining: 17.1s
7:	learn: 0.3182392	total: 1.51s	remaining: 17.3s
8:	learn: 0.3162274	total: 1.68s	remaining: 17s


...

61:	learn: 0.2784401	total: 11.5s	remaining: 7.07s
62:	learn: 0.2782567	total: 11.7s	remaining: 6.88s
63:	learn: 0.2780851	total: 11.9s	remaining: 6.69s
64:	learn: 0.2778558	total: 12.1s	remaining: 6.51s
65:	learn: 0.2776346	total: 12.3s	remaining: 6.33s
66:	learn: 0.2774398	total: 12.5s	remaining: 6.14s
67:	learn: 0.2772922	total: 12.7s	remaining: 5.95s
68:	learn: 0.2770824	total: 12.8s	remaining: 5.77s
69:	learn: 0.2768844	total: 13s	remaining: 5.59s
70:	learn: 0.2767051	total: 13.2s	remaining: 5.4s
71:	learn: 0.2765661	total: 13.4s	remaining: 5.21s
72:	learn: 0.2764311	total: 13.6s	remaining: 5.02s
73:	learn: 0.2762550	total: 13.8s	remaining: 4.83s
74:	learn: 0.2761093	total: 13.9s	remaining: 4.64s
75:	learn: 0.2760265	total: 14.1s	remaining: 4.44s
76:	learn: 0.2758008	total: 14.3s	remaining: 4.26s
77:	learn: 0.2755915	total: 14.4s	remaining: 4.07s
78:	learn: 0.2754017	total: 14.6s	remaining: 3.89s
79:	learn: 0.2752582	total: 14.8s	remaining: 3.71s
80:	learn: 0.2750940	total: 15s	re

23:	learn: 0.2885002	total: 4.51s	remaining: 14.3s
24:	learn: 0.2879154	total: 4.72s	remaining: 14.2s
25:	learn: 0.2876594	total: 4.91s	remaining: 14s
26:	learn: 0.2872342	total: 5.11s	remaining: 13.8s
27:	learn: 0.2868684	total: 5.33s	remaining: 13.7s
28:	learn: 0.2864937	total: 5.54s	remaining: 13.6s
29:	learn: 0.2863661	total: 5.7s	remaining: 13.3s
30:	learn: 0.2860945	total: 5.87s	remaining: 13.1s
31:	learn: 0.2854351	total: 6.06s	remaining: 12.9s
32:	learn: 0.2849987	total: 6.24s	remaining: 12.7s
33:	learn: 0.2847114	total: 6.42s	remaining: 12.5s
34:	learn: 0.2845117	total: 6.6s	remaining: 12.3s
35:	learn: 0.2842103	total: 6.8s	remaining: 12.1s
36:	learn: 0.2840127	total: 6.99s	remaining: 11.9s
37:	learn: 0.2837832	total: 7.18s	remaining: 11.7s
38:	learn: 0.2834143	total: 7.39s	remaining: 11.6s
39:	learn: 0.2831734	total: 7.58s	remaining: 11.4s
40:	learn: 0.2829446	total: 7.76s	remaining: 11.2s
41:	learn: 0.2826724	total: 7.96s	remaining: 11s
42:	learn: 0.2824622	total: 8.15s	rema

85:	learn: 0.2744034	total: 16s	remaining: 2.6s
86:	learn: 0.2742358	total: 16.2s	remaining: 2.42s
87:	learn: 0.2740270	total: 16.4s	remaining: 2.24s
88:	learn: 0.2737732	total: 16.6s	remaining: 2.05s
89:	learn: 0.2736125	total: 16.8s	remaining: 1.87s
90:	learn: 0.2734497	total: 17s	remaining: 1.68s
91:	learn: 0.2732573	total: 17.2s	remaining: 1.49s
92:	learn: 0.2730942	total: 17.3s	remaining: 1.3s
93:	learn: 0.2729554	total: 17.5s	remaining: 1.12s
94:	learn: 0.2727661	total: 17.7s	remaining: 933ms
95:	learn: 0.2726558	total: 17.9s	remaining: 746ms
96:	learn: 0.2725087	total: 18.1s	remaining: 559ms
97:	learn: 0.2723351	total: 18.2s	remaining: 372ms
98:	learn: 0.2721448	total: 18.5s	remaining: 187ms
99:	learn: 0.2720128	total: 18.7s	remaining: 0us
0:	learn: 0.4735912	total: 184ms	remaining: 18.2s
1:	learn: 0.3899352	total: 360ms	remaining: 17.6s
2:	learn: 0.3502842	total: 540ms	remaining: 17.5s
3:	learn: 0.3286013	total: 728ms	remaining: 17.5s
4:	learn: 0.3174798	total: 898ms	remaining:

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [31:42<00:00, 190.22s/it]

sum_: (10, 510414)
Bagging train/test accuracies 0.866/0.859


In [11]:
from joblib import dump
dump(class_, "class_.txt")  

['class_.txt']

In [ ]:
temp = stats.mode(sum_,axis=0)

In [ ]:
fin_res = temp.mode.reshape(-1)

In [43]:
sub_res = stats.mode(sum_test,axis=0)
fin_res = sub_res.mode.reshape(-1)

In [47]:
print(len(np.where(fin_res==0)[0]),len(np.where(fin_res==1)[0]),len(np.where(fin_res==2)[0]))

172078 3038 941


In [12]:
i = 0
for cla in tqdm(class_):
    temp_res = cla.predict_proba(X_test)
    if (i==0):
        fin = temp_res
        i=2
    else:
        #fin = np.vstack((fin,temp_res))
        fin = fin+temp_res

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:18<00:00,  1.65s/it]


In [15]:
ah = fin.copy()

In [16]:
fin = ah.copy()
fin = fin/fin.sum(axis=1).reshape(-1,1)

In [17]:
len(np.where(fin[:,0]>0.62)[0])

169092

In [18]:
len(np.where(fin[:,1]>0.34)[0])

5125

In [19]:
len(np.where(fin[:,2]>0.3)[0])

1594

In [20]:
a = (np.zeros((176057,1))-1)

In [21]:
#len(np.where(fin[:,2]>0.33)[0])
a[np.where(fin[:,2]>0.3)[0]]=3
fin[np.where(fin[:,2]<=0.3)[0],2]=0

In [22]:
a[np.where(fin[:,0]>0.62)[0]]=1
fin[np.where(fin[:,0]<=0.62)[0],0]=0

In [23]:
fin

array([[0.96051194, 0.02978834, 0.        ],
       [0.98023143, 0.01431899, 0.        ],
       [0.99527299, 0.00407563, 0.        ],
       ...,
       [0.99524147, 0.00182974, 0.        ],
       [0.98270571, 0.01251756, 0.        ],
       [0.96592013, 0.03212243, 0.        ]])

In [24]:
a = np.where(a==-1,2,a)

In [25]:
len(np.where(a==1)[0])

169092

In [26]:
len(np.where(a==2)[0])

5371

In [27]:
len(np.where(a==3)[0])

1594

In [29]:
lgb = test[['link', 'current_slice_id', 'future_slice_id', 'label']]

In [30]:
lgb['label'] = a

In [31]:
lgb.to_csv('2020_11_22_1.csv', index=False, encoding='utf8')

In [34]:
test1 = pd.read_csv('time_slice/one_day.txt')
test2 = pd.read_csv('time_slice/two_day.txt')
test = pd.concat((test1,test2))

In [36]:
#test['is_night'] = test['future_slice_id'].apply(lambda x: 0 if (0< x <= 172) | (x>=690) else 1)
test = test.merge(attr, on='link', how='left')
test = test.merge(pro, on='link', how='left')

In [37]:
X_test = (test.loc[:,use_cols])
y_test = (test['label'])

In [38]:
i = 0
for cla in tqdm(class_):
    temp_res = cla.predict(X_test)
    if (i==0):
        fin = temp_res
        i=2
    else:
        fin = np.vstack((fin,temp_res))

100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [00:35<00:00,  3.23s/it]


In [39]:
temp = stats.mode(fin,axis=0)

In [40]:
fin_res = temp.mode.reshape(-1)

In [42]:
print(classification_report(y_test,fin_res,digits=4))
report = f1_score(y_test,fin_res, average=None)
print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)

              precision    recall  f1-score   support

           0     0.9794    0.9903    0.9848    339506
           1     0.5245    0.3666    0.4315      9788
           2     0.5339    0.3709    0.4377      2820

    accuracy                         0.9680    352114
   macro avg     0.6793    0.5759    0.6180    352114
weighted avg     0.9631    0.9680    0.9651    352114

Score:  0.5459204974192585


In [40]:
lgb = test[['link', 'current_slice_id', 'future_slice_id', 'label']]

In [42]:
lgb['label']=fin_res+1

In [43]:
lgb.to_csv('2020_11_17.csv', index=False, encoding='utf8')

## 34万bagging